### Quasi-constant features
Quasi-constant features are those that show the same value for the great majority of the observations of the dataset. In general, these features provide little, if any, information that allows a machine learning model to discriminate or predict a target. But there can be exceptions. So you should be careful when removing these type of features.

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.feature_selection import VarianceThreshold

In [2]:
df = pd.read_csv('../data/dataset_1.csv')

In [3]:
df.head(2)

,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,...,var_287,var_288,var_289,var_290,var_291,var_292,var_293,var_294,var_295,var_296,var_297,var_298,var_299,var_300,target
0,0,0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0,0,...,0,0.0,0,0.0,0,0.0,0,0,0,0,0,0,0.0,0.0,0
1,0,0,0.0,3.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0,3,...,0,0.0,0,0.0,0,0.0,0,0,0,0,0,0,0.0,0.0,0


In [4]:
df.shape

(50000, 301)

In [6]:
# separate dataset into train and test

X_train, X_test, y_train, y_test = train_test_split(
    df.drop(labels=['target'], axis=1), # drop the target
    df['target'], # just the target
    test_size=0.3,
    random_state=0)

X_train.shape, X_test.shape

((35000, 300), (15000, 300))

# Removing the constants 


In [7]:

constant_features = [
    feat for feat in X_train.columns if X_train[feat].std() == 0
]

X_train.drop(labels=constant_features, axis=1, inplace=True)
X_test.drop(labels=constant_features, axis=1, inplace=True)

X_train.shape, X_test.shape

((35000, 266), (15000, 266))

## Remove quasi-constant features
### Using the VarianceThreshold from sklearn
The VarianceThreshold from sklearn provides a simple baseline approach to feature selection. It removes all features which variance doesn’t meet a certain threshold. By default, it removes all zero-variance features

Here, we will change the default threshold to remove quasi-constant features, or, I should better say, features with low-variance:

In [9]:
# Features with a training-set variance lower than this threshold will be removed. The default is to keep all features \
# with non-zero variance i.e. remove the features that have the same value in all samples.
# 1 percent
sel = VarianceThreshold(threshold=0.01)
sel.fit(X_train) # Fit find the features with low variance

VarianceThreshold(threshold=0.01)

In [10]:
# get_support is a boolean vector that indicates which features are retained
# if we sum over get_support, we get the number of features that are not quasi-constant

sum(sel.get_support())

215

In [15]:
# GEtting only quasi-constant features
quasi_constant = X_train.columns[~sel.get_support()]
len(quasi_constant)

51

We can see that 51 columns / variables are almost constant. This means that 51 variables show predominantly one value for the majority of observations of the training set. Let's explore a few if these variables below.

In [16]:
quasi_constant

Index(['var_1', 'var_2', 'var_7', 'var_9', 'var_10', 'var_19', 'var_28',
       'var_36', 'var_43', 'var_45', 'var_53', 'var_56', 'var_59', 'var_66',
       'var_67', 'var_69', 'var_71', 'var_104', 'var_106', 'var_116',
       'var_133', 'var_137', 'var_141', 'var_146', 'var_177', 'var_187',
       'var_189', 'var_194', 'var_197', 'var_198', 'var_202', 'var_218',
       'var_219', 'var_223', 'var_233', 'var_234', 'var_235', 'var_245',
       'var_247', 'var_249', 'var_250', 'var_251', 'var_256', 'var_260',
       'var_267', 'var_274', 'var_282', 'var_285', 'var_287', 'var_289',
       'var_298'],
      dtype='object')

In [14]:
# percent of obeservations showing each of the different values of the variables

X_train['var_1'].value_counts()

0    34987
3        7
6        5
9        1
Name: var_1, dtype: int64

In [18]:
X_train['var_1'].value_counts()/np.float64(len(X_train))

0    0.999629
3    0.000200
6    0.000143
9    0.000029
Name: var_1, dtype: float64

We can see that more than > 99% of the observations show one value 0. Therefore, this feature is almost consider as constant.


In [19]:
X_train['var_1'].var()

0.009254468495021385

In [21]:
X_train['var_1'].var() <0.01

True

In [20]:
1-X_train['var_1'].var()

0.9907455315049786

In [24]:
# Lets check for another variable for which var is greater than threshold
X_train['var_3'].value_counts()

0.0000         34987
207901.3365        1
15028.0560         1
25905.4866         1
35685.9459         1
3583.3941          1
52105.7901         1
86718.0000         1
861.0900           1
2641.0164          1
5209.9500          1
10281.6000         1
12542.3100         1
27.3000            1
Name: var_3, dtype: int64

In [26]:
X_train['var_3'].value_counts()/np.float64(len(X_train))

0.0000         0.999629
207901.3365    0.000029
15028.0560     0.000029
25905.4866     0.000029
35685.9459     0.000029
3583.3941      0.000029
52105.7901     0.000029
86718.0000     0.000029
861.0900       0.000029
2641.0164      0.000029
5209.9500      0.000029
10281.6000     0.000029
12542.3100     0.000029
27.3000        0.000029
Name: var_3, dtype: float64

In [28]:
X_train['var_3'].var()

1598135.0964543768

In [27]:
X_train['var_3'].var() <0.01

False

We can then remove the quasi-constant features utilizing the transform() method from the VarianceThreshold. Remember that this returns a NumPy array without feature names, so if we want a dataframe we need to reconstitute it.

In [29]:
# capture feature names

feat_names = X_train.columns[sel.get_support()]

In [30]:
#remove the quasi-constant features

X_train = sel.transform(X_train)
X_test = sel.transform(X_test)

X_train.shape, X_test.shape

((35000, 215), (15000, 215))

By removing constant and almost constant features, we reduced the feature space from 300 to 215. This means, that 85 features were removed from this dataset. Almost a third!!

In [31]:
# trasnform the array into a dataframe

X_train = pd.DataFrame(X_train, columns=feat_names)
X_test = pd.DataFrame(X_test, columns=feat_names)

X_test.head()

,var_3,var_4,var_5,var_6,var_8,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_20,var_21,...,var_279,var_280,var_281,var_283,var_284,var_286,var_288,var_290,var_291,var_292,var_293,var_295,var_296,var_299,var_300
0,0.0,2.79,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,13.6500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,2.94,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,48763.8912,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
3,0.0,2.76,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,86.4000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,2.94,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,2.9700,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Without Using the Package

First, I will separate the dataset into train and test and remove the constant features again. Then, I will provide an alternative method to find out quasi-constant features.

This method, as opposed to the VarianceThreshold, can be used for both numerical and categorical variables.

In [33]:
# separate train and test
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(labels=['target'], axis=1),
    df['target'],
    test_size=0.3,
    random_state=0)

# remove constant features
# using the code from the previous lecture

constant_features = [
    feat for feat in X_train.columns if X_train[feat].std() == 0
]

X_train.drop(labels=constant_features, axis=1, inplace=True)
X_test.drop(labels=constant_features, axis=1, inplace=True)

X_train.shape, X_test.shape

((35000, 266), (15000, 266))

In [34]:
quasi_features = [
    feat for feat in X_train.columns if X_train[feat].var() < 0.01
]

In [35]:
len(quasi_features)

51

In [43]:
(X_train['var_1'].value_counts() / np.float64(
        len(X_train))).sort_values(ascending=False).values[0]

0.9996285714285714

In [45]:
# create an empty list
quasi_constant_feat = []

# iterate over every feature
for feature in X_train.columns:

    # find the predominant value, that is the value that is shared
    # by most observations
    predominant = (X_train[feature].value_counts() / np.float64(
        len(X_train))).sort_values(ascending=False).values[0]

    # evaluate the predominant feature: do more than 99% of the observations
    # show 1 value?
    if predominant > 0.998:
        
        # if yes, add the variable to the list
        quasi_constant_feat.append(feature)

len(quasi_constant_feat)

108

Our method was a bit more aggressive than VarianceThreshold from sklearn with the threshold that we selected above. It found 108 features that show predominantly 1 value for the majority of the observations.

In [46]:
quasi_constant_feat

['var_1',
 'var_2',
 'var_3',
 'var_6',
 'var_7',
 'var_9',
 'var_10',
 'var_11',
 'var_12',
 'var_14',
 'var_16',
 'var_20',
 'var_24',
 'var_28',
 'var_32',
 'var_34',
 'var_36',
 'var_39',
 'var_40',
 'var_42',
 'var_43',
 'var_45',
 'var_48',
 'var_53',
 'var_56',
 'var_59',
 'var_60',
 'var_65',
 'var_66',
 'var_67',
 'var_69',
 'var_71',
 'var_72',
 'var_73',
 'var_77',
 'var_78',
 'var_90',
 'var_95',
 'var_98',
 'var_102',
 'var_104',
 'var_106',
 'var_111',
 'var_115',
 'var_116',
 'var_124',
 'var_125',
 'var_126',
 'var_129',
 'var_130',
 'var_133',
 'var_136',
 'var_138',
 'var_141',
 'var_142',
 'var_146',
 'var_149',
 'var_150',
 'var_151',
 'var_153',
 'var_159',
 'var_183',
 'var_184',
 'var_187',
 'var_189',
 'var_197',
 'var_202',
 'var_204',
 'var_210',
 'var_211',
 'var_216',
 'var_217',
 'var_219',
 'var_221',
 'var_223',
 'var_224',
 'var_228',
 'var_233',
 'var_234',
 'var_235',
 'var_236',
 'var_237',
 'var_239',
 'var_243',
 'var_245',
 'var_246',
 'var_247',
 

In [47]:
# select one feature from the list

quasi_constant_feat[2:5]

['var_3', 'var_6', 'var_7']

In [49]:
X_train['var_3'].value_counts() / np.float64(len(X_train))

0.0000         0.999629
207901.3365    0.000029
15028.0560     0.000029
25905.4866     0.000029
35685.9459     0.000029
3583.3941      0.000029
52105.7901     0.000029
86718.0000     0.000029
861.0900       0.000029
2641.0164      0.000029
5209.9500      0.000029
10281.6000     0.000029
12542.3100     0.000029
27.3000        0.000029
Name: var_3, dtype: float64

Our method was a bit more aggressive than VarianceThreshold from sklearn with the threshold that we selected above. It found 108 features that show predominantly 1 value for the majority of the observations.

In [50]:
# finally, let's drop the quasi-constant features:

X_train.drop(labels=quasi_constant_feat, axis=1, inplace=True)
X_test.drop(labels=quasi_constant_feat, axis=1, inplace=True)

X_train.shape, X_test.shape

((35000, 158), (15000, 158))